This is the **7th** Notebook in the clustering pipeline. It shows you how to evaluate the final clustering solution on outcomes and inputs. This solution could be chosen by hand, by using the NBclust pipeline or another method (like bootstrap). **PATRICIA THIS COULD BE USEFUL FOR WHICHEVER SOLUTION YOU DECIDE TO USE**.

Use <u>***pappas_tadam***</u> virtual environment.

In [ ]:
# Set this to whatever directory GoodCopy is in

home_dir = "/home/l/lungboy/tadam/Project/"

# Importing Packages and Data

In [ ]:
# Importing packages and functions

import numpy as np
import pandas as pd
import umap.umap_ as umap
import matplotlib.pyplot as plt
import sys

sys.path.append(home_dir + 'GoodCopy/Functions')

import FunctionsOOPGood as func

In [ ]:
# Importing Datasets

data = func.DataSet(empty=True)
data.open_DataSet(home_dir + "GoodCopy/Objects/data_object")

data_matched = func.MatchedDataSet(empty=True)
data_matched.open_MatchedDataSet(home_dir + "GoodCopy/Objects/matched_data_saved")

In [ ]:
# Importing Labels

agglo_labels = [pd.read_csv(home_dir + "GoodCopy/EnsembleResults/{}_agglo_labels.csv".format(i), index_col= 0) for i in range(2,10)]
all_labels = [pd.read_csv(home_dir + "GoodCopy/EnsembleResults/{}_all_labels.csv".format(i), index_col= 0) for i in range(2,10)]
hdb_labels = [pd.read_csv(home_dir + "GoodCopy/EnsembleResults/{}_hdb_labels.csv".format(i), index_col= 0) for i in range(2,10)]
kmedoids_labels = [pd.read_csv(home_dir + "GoodCopy/EnsembleResults/{}_kmedoids_labels.csv".format(i), index_col= 0) for i in range(2,10)]
snf_hdb_labels = [pd.read_csv(home_dir + "GoodCopy/EnsembleResults/{}_snf_hdb_labels.csv".format(i), index_col= 0) for i in range(2,10)]
snf_spectral_labels =[pd.read_csv(home_dir + "GoodCopy/EnsembleResults/{}_snf_spectral_labels.csv".format(i), index_col= 0) for i in range(2,10)]

labels = agglo_labels + all_labels + hdb_labels + kmedoids_labels + snf_hdb_labels + snf_spectral_labels

In [ ]:
# CHOOSE YOUR FINAL LABEL SOLUTION HERE. Should be the form of a 1D array (ie list, numpy array, 
# pandas series, etc).

final_solution = np.ravel(hdb_labels[6])

# Statistical Testing on Clusters
Statistical Testing on clusters of final solution.

In [ ]:
# Generate Statistical Tests

import warnings

# Turn off all warnings
warnings.filterwarnings("ignore")

eval_output = data.EvaluateClusters(data = "output", labels = final_solution, range_type="quantile", range_level = 0.5)

eval_input = data.EvaluateClusters(data = "input", labels = final_solution)

# Reset warnings to their default behavior (optional)
warnings.resetwarnings()

## Outcome Variables

### Statistical Testing, only Significant Outcomes are displayed

In [ ]:
# View significant output variables. The first dataframe contains categorical variables analyzes with chisquare
# and the second contains continous analyzed with ANOVA.

print("\n\033[1mSignificant Categorical Outcomes\n")
display(eval_output[0][eval_output[0]["pvalue"] < 0.05])
print("\n\033[1mSignificant Continuous Outcomes\n")
display(eval_output[1][eval_output[1]["pvalue"] < 0.05])
#display(eval_output[1][(eval_output[1]["welch pvalue"] <= 0.05) & (eval_output[1]["valid"] == False) | (eval_output[1]["pvalue"] <= 0.05) & (eval_output[1]["valid"] == True)])

### Value counts for Statistically Significant Categorical/Binary Outcome Variables

In [ ]:
# View Counts for Each category for categorical variables. This one shows output variables.

sig_vars_cat = eval_output[0][eval_output[0]["pvalue"] < 0.05].index.tolist()
#sig_vars_cat.append("f34_pet")

#count_df_output_cat = pd.DataFrame(index = np.unique(final_solution))

for i in sig_vars_cat:
    
    counts = func.DataSet.get_val_counts(data = data.outcome_bin_cat, labels = final_solution , var = i)
    
    print("\n\033[1m",i,"\033[0m\n")
    for j in range(len(counts)):
        print(j,"\t",", ".join([f"{key}: {value}" for d in counts[j] for key, value in d.items()]))

### Mean and Median Statistics for Statistically Significant Continuous/Ordinal Variables

In [ ]:
# View means for each continouos variable. This one shows output variables.

sig_vars_cont = eval_output[1][eval_output[1].pvalue < 0.05].index

mean_df_output_cont = pd.DataFrame(index = np.unique(final_solution))
median_df_output_cont = pd.DataFrame(index = np.unique(final_solution))

for i in sig_vars_cont:
    
    counts = func.DataSet.get_cluster_sum_var(data = data.outcome_cont_ord, labels = final_solution , var = i)
    
    mean_df_output_cont[i] = [x[0] for x in counts]
    median_df_output_cont[i] = [x[1] for x in counts]
    
print("\n\033[1mMean\n")
display(mean_df_output_cont)
print("\n\033[1mMedian\n")
median_df_output_cont

## Input Variables

### Statistical Testing, only Significant Inputs are displayed

In [ ]:
# View significant input variables. The first dataframe contains categorical variables analyzes with chisquare
# and the second contains continous analyzed with ANOVA.

print("\n\033[1mSignificant Categorical Inputs\n")
display(eval_input[0][eval_input[0]["pvalue"] < 0.05])
print("\n\033[1mSignificant Continuous Inputs\n")
display(eval_input[1][eval_input[1].pvalue < 0.05])

### Value counts for Statistically Significant Categorical/Binary Input Variables

In [ ]:
# View Counts for Each category for categorical variables. This one shows output variables.

sig_vars_cat = eval_input[0][eval_input[0]["pvalue"] < 0.05].index.tolist() 

count_df_output_cat = pd.DataFrame(index = np.unique(final_solution))

for i in sig_vars_cat:
    
    counts = func.DataSet.get_val_counts(data = data.input_data_unscaled, labels = final_solution , var = i)
    
    print("\n\033[1m",i,"\033[0m\n")
    for j in range(len(counts)):
        print(j,"\t",", ".join([f"{key}: {value}" for d in counts[j] for key, value in d.items()]))

### Mean and Median Statistics for Statistically Significant Continuous/Ordinal Input Variables

In [ ]:
# View Counts for Each category for categorical variables. This one shows output variables.

sig_vars_cont = eval_input[1][eval_input[1].pvalue < 0.05].index

mean_df_input_cont = pd.DataFrame(index = np.unique(final_solution))
median_df_input_cont = pd.DataFrame(index = np.unique(final_solution))

for i in sig_vars_cont:
    
    counts = func.DataSet.get_cluster_sum_var(data = data.input_data_unscaled, labels = final_solution , var = i)

    mean_df_input_cont[i] = [x[0] for x in counts]
    median_df_input_cont[i] = [x[1] for x in counts]
    
    mean_df_input_cont = mean_df_input_cont.copy()
    median_df_input_cont = median_df_input_cont.copy()
    
print("\n\033[1mMean\n")
display(mean_df_input_cont)
print("\n\033[1mMedian\n")
median_df_input_cont

### Interpretable classifier results for Input Variables

In [ ]:
input_interpretable_vars = data.random_forest_importance(labels = final_solution, n_class= len(np.unique(final_solution)))

In [ ]:
print("\n\033[1mImportant Input Variables in Differentiating between Clusters\n")
input_interpretable_vars.head(15)

## Biomarker Variables

### Interpretable classifier results for Biomarker Variables

In [ ]:
# Prepping data by filling in NAs

df = data.bio_data.copy()

mean_values = df[~np.isinf(df)].mean()

# Step 2: Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 3: Fill NaN values with the calculated mean
df.fillna(mean_values, inplace=True)

In [ ]:
bio_interpretable_vars = data.random_forest_importance(data = df,labels = final_solution, n_class= len(np.unique(final_solution)))

eval_bio = data.EvaluateClusters(labels = final_solution, data="bio")

In [ ]:
print("\n\033[1mSignificant Continuous Biomarker Variables\n")
display(eval_bio[1][eval_bio[1].pvalue < 0.05])

In [ ]:
print("\n\033[1mImportant Biomarker Variables in Differentiating between Clusters\n")
bio_interpretable_vars.head(15)

# Extra Stuff

sample workflows for potential things you might want to explore in a solution

## Looking at differences between PE and control within one cluster

In [ ]:
# Extract cluster 1 and do random forest importance to see what differentiates the controls from PE

cluster_number = 5

data_temp = data.input_data_unscaled.copy()
data_temp["labels"] = final_solution
data_temp["PE"] = data.pe_labels.copy()

data_temp = data_temp[data_temp["labels"] == cluster_number]

diff = data_temp["PE"]

res = data.random_forest_importance(data=data_temp.drop(columns=["labels","PE"]),labels=diff,n_class = 2)

res1 = data.EvaluateClusters(labels = diff, data="input", subset = list(data_temp.index))

In [ ]:
print("\n\033[1mImportant Input Variables in Differentiating between PE and Control within Cluster {}\n".format(cluster_number))
res.head(10)

In [ ]:
print("\n\033[1mSignificant Categorical Inputs between PE and control in Cluster {}\n".format(cluster_number))
display(res1[0][res1[0].pvalue < 0.05])
print("\n\033[1mSignificant Continuous Inputs between PE and control in Cluster {}\n".format(cluster_number))
display(res1[1][res1[1].pvalue < 0.05])